In [ ]:
# Setting up
from __future__ import print_function, division
import os
import numpy as np
import torch
import pandas as pd
from skimage import io, transform

import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from data_loading import *

train = SkinLesionDataset(csv_file='../datasets/ISIC-2017_Training_Part3_GroundTruth.csv',
                                    root_dir='../datasets/ISIC-2017_Training_Data/', transform = transforms.ToTensor())
validation  = SkinLesionDataset(csv_file='../datasets/ISIC-2017_Validation_Part3_GroundTruth.csv',
                                    root_dir='../datasets/ISIC-2017_Validation_Data/', transform = transforms.ToTensor())
test = SkinLesionDataset(csv_file='../datasets/ISIC-2017_Test_v2_Part3_GroundTruth.csv',
                                    root_dir='../datasets/ISIC-2017_Test_v2__Data/', transform = transforms.ToTensor())

In [ ]:
# Vizualizing some images

for i in range(len(face_dataset)):
    sample = face_dataset[i]
    #print(sample['image'])
    print(i, sample['image'].shape, sample['class1'], sample['class2'])

    ax = plt.subplot(1, 4, i + 1)
    plt.tight_layout()
    ax.set_title('Sample #{}'.format(i))
    ax.axis('off')
    #show_landmarks(sample['image'])

    #plt.show()
    if i == 3:
        plt.show()
        break
    plt.pause(10)
